<a href="https://colab.research.google.com/github/Bob-Gohardani/nlp-ml/blob/main/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reccomendation engine written in pure Python, only external library is numpy.

In [ ]:
# Download Data

import csv
import random
import math
import operator
import requests
movies = {}


def downaload_dataset():
    #downloading data from API:

    api_key = "24370f953f5f3f70890d54a4705b8ff4"

    with open('movies.csv', 'rU') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        for row in reader:
            resp = requests.get("https://api.themoviedb.org/3/movie/"+ row[1] +"?api_key="+api_key)
            json_resp = resp.json()
            # Here I only use the first genre of each movie mentioned
            movies[row[0]] = [
                              str(row[0]),
                              str(json_resp["budget"]),
                              str(json_resp["genres"][0]["id"]),
                              str(json_resp["runtime"]),
                              str(json_resp["release_date"].split('-')[0]),
                             ]

    with open("movies_1.csv", 'wb') as myfile:
        wr = csv.writer(myfile)
        for key, value in movies.iteritems():
            wr.writerow(value)

In [ ]:
# Pearson’s Correlation

from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr

def pearson_correl():
    data1 = [item[0] for item in DataSet]
    data8 = [item[7] for item in DataSet]
    # calculate Pearson's correlation


    corr, _ = pearsonr(data1, data8)
    print('Pearsons correlation: %.3f' % corr)

In [ ]:
# nearest neighbour reccomendation
import csv
import random
import math
import operator
import numpy as np


class NearesteighbourEngine:
    
    DataSet = {}
    movies = {}
    validation_sets = []
    training_sets = []
    l1 = []


    # Data Normalization:
    def normalizeData():
        with open("movies_1.csv", 'rb') as myfile:
            reader = csv.reader(myfile)
            for row in reader:
                x = [float(y) for y in row[1:]]
                movies[row[0]] = x

        dct = {}
        for i in range(4):
            dct[i] = []

        for k, v in movies.iteritems():
            for j in range(4):
                dct[j].append(v[j])

        for key in dct:
            high = max(dct[key])
            low = min(dct[key])
            dct[key] = []
            dct[key].append(low)
            dct[key].append(high)

        for k, v in movies.iteritems():
            for i in range(len(v)):
                i = int(i)
                y = (2 * (v[i] - dct[i][0]) / (dct[i][1] - dct[i][0])) - 1
                movies[k][i] = y

        # download data from different csv files and mix them together in dataset
        with open('train.csv', 'rb') as f:
            reader = csv.reader(f)
            l1 = list(reader)


    for l in l1:
        N = [float(x) for x in l[0].split(';')]
        features = movies[str(int(N[2]))]
        for j in features:
            N.append(float(j))

        if str(int(N[1])) not in DataSet.keys():
            DataSet[str(int(N[1]))] = [N[2:]]
        else:
            DataSet[str(int(N[1]))].append(N[2:])

    #looping through k from 1 to 10 to find proper neighbourhood:
    def euclideanDistance(inst1, inst2):
        distance = 0
        for x in range(2,6):
            distance += pow((inst1[x] - inst2[x]), 2)
        return math.sqrt(distance)

    def getNeighbors(trainingSet, testInstance, k):
        distances = []
        length = len(testInstance)-1
        for x in range(len(trainingSet)):
            dist = euclideanDistance(testInstance, trainingSet[x])
            distances.append((trainingSet[x], dist))
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
        # find K closest neighbors
        for x in range(k):
            neighbors.append(distances[x][0])
        return neighbors

    # get averge of rating of neighbours
    def getResponse(neighbors):
        sum = 0
        for x in range(len(neighbors)):
            sum += neighbors[x][1]
        return round(sum / len(neighbors))

    def getErrorRate(testSet, predictions):
        sum = 0
        for x in range(len(testSet)):
            sum = abs(float(testSet[x][1]) - predictions[x])
        return (sum/float(len(testSet)))

    errors_hyper = []
    
    def k_selection():
        for k in range(1, 10):
            error_set = []
            for key in DataSet.keys():
                predictions=[]
                person1 = DataSet[key]
                trainingSet = []
                testSet = []
                for i in range(int(len(person1)/10)):
                    x = person1[random.randint(0, len(person1)-1)]
                    testSet.append(x)
                    person1.remove(x)
                trainingSet = person1

                for x in range(len(testSet)):
                    neighbors = getNeighbors(trainingSet, testSet[x], k)
                    result = getResponse(neighbors)
                    predictions.append(result)

                error = getErrorRate(testSet, predictions)
                error_set.append(error)
                errors_hyper.append((k,sum(error_set)/len(error_set)))

        final_k = min(errors_hyper, key = lambda t: t[1])[0]


    def final_test():
        with open('task.csv', 'rb') as f:
            reader = csv.reader(f)
            your_list = list(reader)
        list_1 = []

        for l in your_list:
            N = [x for x in l[0].split(';')]
            user_data = DataSet[N[1]]  # all train data relating to this user
            movie_info = movies[N[2]]  # features related to this movie
            movie_info = [1,1]+movie_info
            neighbors = getNeighbors(user_data, movie_info, final_k)
            result = getResponse(neighbors)
            N[3] = int(result)
            list_1.append(N)

        with open("submission.csv", 'wb') as myfile:
            wr = csv.writer(myfile, delimiter=';')
            for l in list_1:
                wr.writerow(l)

In [ ]:
# tree with prunning

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]


max_Depth = 5;


def unique_vals(rows, col):
    return set([row[col] for row in rows])

def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    # in both conditions returns true
    return isinstance(value, int) or isinstance(value, float)


# A Question is used to partition a dataset
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))


def partition(rows, question):
    """Partitions a dataset.
    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows




def gini(rows):
    #Calculate the Gini Impurity for a list of rows.
    counts = class_counts(rows) # Counts the number of each type of example in a dataset
    impurity = 1

    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity



def info_gain(left, right, current_uncertainty):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of two child nodes.
    """
    p = float(len(left) / len(left) + len(right))
    # whatever probabilty of left is, right is 1- p_left
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)


def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""

    best_gain = 0 # keep track of best information gain
    best_question = None #  keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns

    for col in range(n_features): # for each feature
        values = set([row[col] for row in rows]) # unique values in the column

        for val in values: # for each value:
            question = Question(col, val)

            # try splitting the dataset:
            true_rows, false_rows = partition(rows, question)
            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

class leaf:
    """A Leaf node classifies data.
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)



class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


def build_tree(rows, depth):
    gain, question = find_best_split(rows)

    # STOP POINT OF RECURSIVE TREE
    if gain == 0 or depth >= max_Depth:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows, depth+1)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows, depth+1)

    # Return a Question node. This records the best feature / value to ask at this point, as well as the branches to follow dependingo on the answer. (branch with two more nodes to follow it)
    return Decision_Node(question, true_branch, false_branch)

# we use the calissify function to make classification for each row  of dataset via the decision_tree we made
def classify(row, node):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    # you will repeat this loop until you reach leaf
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


def calculate_accuracy(data):
    sum = 0
    for row in training_data:
        if row[-1] == classify(row, data):
            sum += 1
    return float( sum / len(data) )


# treat depth as hyper-parameter for tree pruning:
def prune():
    error_set = []
    for i in range(4, 10):
        max_Depth = i
        testSet = []
        data_temp = training_data.copy()
        for i in range(int(len(data_temp)/5)):
            x = data_temp[str(random.choice(len(data_temp)))]
            testSet.append(x)
            data_temp.remove(x)

        my_tree = build_tree(training_data, 1)
        accur = calculate_accuracy(testSet)
        error_set.append([max_Depth, accur])

    max_Depth = min(error_set, key = lambda t: t[1])[0]
    my_tree = build_tree(training_data, 1)


    # Evaluate
    testing_data = [
        ['Green', 3, 'Apple'],
        ['Yellow', 4, 'Apple'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
    ]

    for row in testing_data:
        classify(row, my_tree)

In [ ]:
# item_item_collab_filter
import csv
import random
import math
import operator
import numpy as np


class ItemItemCollabFilter:

    DataSet = {} # for users
    DataSet2 = {} # for movies
    relations = []
    relations = {}

    def getData():
        with open('train.csv', 'rb') as f:
            reader = csv.reader(f)
            l1 = list(reader)

        for l in l1:
            N = [int(x) for x in l[0].split(';')]

            if str(int(N[1])) not in DataSet.keys():
                DataSet[str(int(N[1]))] = [N[1:]]
            else:
                DataSet[str(int(N[1]))].append(N[1:])

            if str(int(N[2])) not in DataSet2:
                DataSet2[str(int(N[2]))] = [[N[1], N[3]]]
            else:
                DataSet2[str(int(N[2]))].append([N[1], N[3]])


    def movieRelation():
        for k, v in DataSet2.iteritems():
            # averge rating of the movie among all users
            movie_avg =  sum(x[1] for x in v) / float(len(v))
            relations[k] = []
            for key, value in DataSet2.iteritems():
                up = 0
                temp1 = []
                temp2 = []

                if key != k:
                    other_avg =  sum(x[1] for x in value)/ float(len(value))
                    for item1 in v:
                        for item2 in value:
                            if (item1[0] == item2[0]) and (item1[1] !=0 or item2[1] != 0):
                                up += (item1[1]-movie_avg) * (item2[1] - other_avg)

                                temp1.append(item1[1])
                                temp2.append(item2[1])
                                break

                    sum1 = 0
                    sum2 = 0
                    for num in temp1:
                        sum1+=(num - movie_avg)**2
                    for num in temp2:
                        sum2+= (num - other_avg)**2
                    down = (sum1**0.5)*(sum2**0.5)
                    rel = float(up) / down
                    relations[k].append((key, rel))

    #movieRelation()


    def findSimilar(userID, k, movieID):
        user = DataSet[str(userID)]

        # if user has same rating for almost all movies then he will probably have same one for this movie:
        variance = np.var([x[2] for x in user])
        if variance <= 0.6:
            return sum([x[2] for x in user]) / float(len(user))

        # find k closet movies from this user's ratings
        similar = relations[str(movieID)]
        similar = sorted(similar, key=lambda x : x[1], reverse=True)
        similar = similar[:k]
        down1 = 0.01
        up1 = 0
        for tuple in similar:
            l = DataSet[str(userID)]
            for x in l:
                if int(x[1]) == int(tuple[0]):
                    down1 += abs(tuple[1])
                    up1 += tuple[1] * x[2]
                    break
        return round(float(up1)/ down1)

        error_set = []
        for k in range(2, 15):
            testSet = []
            data_temp = DataSet.copy()

            for i in range(int(len(data_temp)/10)):
                x = data_temp[str(random.choice(data_temp.keys()))]
                testSet.append(x)
                del data_temp[str(x[0][0])]

            user_error = []
            for user in testSet:
                temp = [] # errors of all movies of a single user
                for rating in user:
                    rate_guess = findSimilar(user[0][0], k, rating[1])
                    temp.append(abs(rating[2] - rate_guess))
                user_error.append(sum(temp)/len(user))
            error_set.append([k, sum(user_error)/len(user_error)])

        final_k = min(error_set, key = lambda t: t[1])[0]

    def submit_results():
        with open('task.csv', 'rb') as g:
            reader1 = csv.reader(g)
            your_list = list(reader1)

        list_1 = []
        for l in your_list:
            N = [x for x in l[0].split(';')]
            user = DataSet[str(N[1])]

            rate_guess = findSimilar(user[0][0], final_k, N[2])
            N[3] = int(rate_guess)
            list_1.append(N)

        with open("submission.csv", 'wb') as myfile:
            wr = csv.writer(myfile, delimiter=';')
            for l in list_1:
                wr.writerow(l)

In [ ]:
import csv
import random
import math
import operator
import numpy as np

class UserUserCollabFilter:

    DataSet = {}
    listSet = []
    
    def readData():
        with open('train.csv', 'rb') as f:
            reader = csv.reader(f)
            l1 = list(reader)


        for l in l1:
            N = [int(x) for x in l[0].split(';')]

            if str(int(N[1])) not in DataSet.keys():
                DataSet[str(int(N[1]))] = [N[1:]]
            else:
                DataSet[str(int(N[1]))].append(N[1:])
            listSet.append(N[1:])
            

    # [0 :'USERID', 1 :'MOVIEID', 2 :'MOVIE_RATING']
    # find similar users that have already rated this movie:
    def findSimilar(userID, Data, k, movieID):
        user = DataSet[str(userID)]

        # if user has same rating for almost all movies then he will probably have same one for this movie:
        variance = np.var([x[2] for x in user])
        if variance <= 0.6:
            return sum([x[2] for x in user]) / float(len(user))

        similar = []
        user_avg = sum([x[2] for x in user]) / float(len(user))
        for key, v in DataSet.iteritems():
            containsMovie = any(int(movieID) == int(x[1]) for x in v)
            variance1 = np.var([x[2] for x in v])
            if (key != userID) and containsMovie and (variance1 > 1):
                    avg = sum([x[2] for x in v]) / float(len(v))
                    up = 0
                    temp1 = []
                    temp2 = []
                    for item in user:
                        for item2 in v:
                            if item[1] == item2[1]:
                                up += (item2[2] - avg)*(item[2] - user_avg)

                                temp1.append(item[2])
                                temp2.append(item2[2])
                                break
                    sum1 = 0
                    sum2 = 0

                    # for current user
                    for num in temp1:
                        sum1 += (num - user_avg)**2

                    # for all neighbours
                    for num in temp2:
                        sum2 += (num - avg)**2

                    down = (sum1**0.5)*(sum2**0.5)

                    similarity = up / down
                    similar.append((key,similarity))

        similar = sorted(similar, key=lambda x: x[1], reverse=True)
        similar = similar[:k]
        # rate movie based on what other users said about that movie
        l1 = []
        for tuple in similar:
            neighbour_data = DataSet[str(tuple[0])]
            for x in neighbour_data:
                if int(x[1]) == int(movieID):
                    l1.append([x[2], tuple[1]])
                    break

        down1 = sum([x[1] for x in l1])
        up1 = sum([x[0]*x[1] for x in l1])
        return round(float(up1)/down1)


        def guess_results():
            error_set = []
            for k in range(2, 10):
                testSet = []
                #list = DataSet.keys()
                data_temp = DataSet.copy()

                for i in range(int(len(data_temp)/20)):
                    x = data_temp[str(random.choice(data_temp.keys()))]
                    testSet.append(x)
                    del data_temp[str(x[0][0])]

                user_error = []
                for user in testSet:
                    temp = [] # errors of all movies of a single user
                    for rating in user:
                        rate_guess = findSimilar(user[0][0], data_temp, k, rating[1])
                        temp.append(abs(rating[2] - rate_guess))
                    user_error.append(sum(temp)/len(user))

                error_set.append([k, sum(user_error)/len(user_error)])

            final_k = min(error_set, key = lambda t: t[1])[0]

            with open('task.csv', 'rb') as g:
                reader1 = csv.reader(g)
                your_list = list(reader1)

            list_1 = []
            for l in your_list:
                N = [x for x in l[0].split(';')]
                user = DataSet[str(N[1])]
                rate_guess = findSimilar(user[0][0], DataSet, final_k, N[2])
                N[3] = int(rate_guess)
                list_1.append(N)

            with open("submission.csv", 'wb') as myfile:
                wr = csv.writer(myfile, delimiter=';')
                for l in list_1:
                    wr.writerow(l)

In [ ]:
# K Nearest Neighbour

import csv
import random
import math
import operator

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    # here add normalization to input features and set them to 0-1 range
	with open(filename, 'rb') as csvfile:
	    lines = csv.reader(csvfile)
	    dataset = list(lines)
	    for x in range(len(dataset)-1):
	        for y in range(4):
	            dataset[x][y] = float(dataset[x][y])
	        if random.random() < split:
	            trainingSet.append(dataset[x])
	        else:
	            testSet.append(dataset[x])


def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	split = 0.67
	loadDataset('iris.data', split, trainingSet, testSet)
	print('Train set: ' + repr(len(trainingSet)))
	print('Test set: ' + repr(len(testSet)))
	# generate predictions
	predictions=[]
	k = 3
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		result = getResponse(neighbors)
		predictions.append(result)
		print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')